Setting the test random environment with OpenAI Gym

In [9]:
import gym 
import random

In [10]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

D:\Program_Installation\anaconda\lib\site-packages\gym\envs\registration.py:564: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
D:\Program_Installation\anaconda\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [11]:
states

4

In [12]:
actions

2

In [10]:
episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score +=reward
    print(f"Episode: {episode} score: {score}")

Episode: 1 score: 35.0
Episode: 2 score: 22.0
Episode: 3 score: 31.0
Episode: 4 score: 38.0
Episode: 5 score: 29.0
Episode: 6 score: 11.0
Episode: 7 score: 28.0
Episode: 8 score: 25.0
Episode: 9 score: 11.0
Episode: 10 score: 25.0


In [25]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [26]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [27]:
model = build_model(states,actions)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 11:59 - reward: 1.0000

D:\Program_Installation\anaconda\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   55/10000 [..............................] - ETA: 1:50 - reward: 1.0000

D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is dep

10000/10000 [==============================] - 46s 5ms/step - reward: 1.0000: 0s - rewar - ETA: 0s - rew
104 episodes - episode_reward: 95.606 [9.000, 200.000] - loss: 3.225 - mae: 19.463 - mean_q: 39.343

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 46s 5ms/step - reward: 1.0000ETA: 0s - reward: 
50 episodes - episode_reward: 198.840 [174.000, 200.000] - loss: 7.725 - mae: 39.628 - mean_q: 79.700

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000: 7s - reward: 1.000 - ETA: 7s - reward - ETA: 7s - reward: - ETA: 6s - reward: 1.0 - ETA -  - ETA: 0s - reward: 1.0
50 episodes - episode_reward: 199.400 [189.000, 200.000] - loss: 10.820 - mae: 44.249 - mean_q: 88.801

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 48s 5ms/step - reward: 1.0000  - ETA: 3s - rewa
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 12.926 - mae: 44.117 - mean_q: 88.

In [32]:
scores = dqn.test(env,nb_episodes=100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

Saving the model

In [33]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [35]:
del model
del dqn
del env

NameError: name 'model' is not defined

Reloading the Agent from Memory

In [37]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states,actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [38]:
dqn.load_weights('dqn_weights.h5f')

In [40]:
_ = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
